<a href="https://colab.research.google.com/github/seonaann/Enter-Week-1/blob/main/coconut.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import os
import zipfile
import tensorflow as tf

print("Mounting Google Drive...")
drive.mount('/content/drive')


Mounting Google Drive...
Mounted at /content/drive


In [2]:
zip_file_path = '/content/drive/MyDrive/Useless.zip'
extract_to = '/content/dataset'  # Where to extract

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

In [3]:
for path, dirs, files in os.walk('/content/dataset'):
    print(path)


/content/dataset
/content/dataset/Useless
/content/dataset/Useless/test
/content/dataset/Useless/test/NOT COCONUT TREE
/content/dataset/Useless/test/COCONUT TREE
/content/dataset/Useless/train
/content/dataset/Useless/train/NOT COCONUT TREE
/content/dataset/Useless/train/COCONUT TREE


In [9]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import EfficientNetV2B2
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import os

# ✅ Define paths (your dataset location)
base_dir = "/content/dataset/Useless"
train_dir = os.path.join(base_dir, "train")
test_dir = os.path.join(base_dir, "test")

# ✅ Image size and batch size for EfficientNetV2B2
IMG_SIZE = (260, 260)
BATCH_SIZE = 16

# ✅ ImageDataGenerator with EfficientNet preprocessing
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    horizontal_flip=True,
    rotation_range=15,
    zoom_range=0.1
)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# ✅ Load images from directory
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

# ✅ Load EfficientNetV2B2 base model
base_model = EfficientNetV2B2(
    include_top=False,
    weights='imagenet',
    input_shape=(260, 260, 3)
)
base_model.trainable = False  # Freeze the base layers initially

# ✅ Add custom classifier on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)

# ✅ Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# ✅ Define callbacks
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ReduceLROnPlateau(patience=2, factor=0.2, verbose=1),
    ModelCheckpoint("best_model.h5", save_best_only=True)
]

# ✅ Train the model
history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=20,
    callbacks=callbacks
)

# ✅ Evaluate the model
loss, acc = model.evaluate(test_data)
print(f"\n✅ Final Test Accuracy: {acc * 100:.2f}%")



Found 160 images belonging to 2 classes.
Found 40 images belonging to 2 classes.
35839040/35839040 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.6245 - loss: 0.5999

10/10 ━━━━━━━━━━━━━━━━━━━━ 90s 6s/step - accuracy: 0.6353 - loss: 0.5891 - val_accuracy: 0.8500 - val_loss: 0.2877 - learning_rate: 0.0010
Epoch 2/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9401 - loss: 0.2089

10/10 ━━━━━━━━━━━━━━━━━━━━ 44s 5s/step - accuracy: 0.9410 - loss: 0.2073 - val_accuracy: 0.9500 - val_loss: 0.1279 - learning_rate: 0.0010
Epoch 3/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 41s 4s/step - accuracy: 0.9251 - loss: 0.1793 - val_accuracy: 0.9250 - val_loss: 0.1399 - learning_rate: 0.0010
Epoch 4/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9603 - loss: 0.0859

10/10 ━━━━━━━━━━━━━━━━━━━━ 41s 4s/step - accuracy: 0.9599 - loss: 0.0869 - val_accuracy: 0.9500 - val_loss: 0.1003 - learning_rate: 0.0010
Epoch 5/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9803 - loss: 0.0739

10/10 ━━━━━━━━━━━━━━━━━━━━ 40s 4s/step - accuracy: 0.9798 - loss: 0.0744 - val_accuracy: 0.9750 - val_loss: 0.0930 - learning_rate: 0.0010
Epoch 6/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 40s 4s/step - accuracy: 0.9696 - loss: 0.1180 - val_accuracy: 0.9250 - val_loss: 0.1227 - learning_rate: 0.0010
Epoch 7/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9811 - loss: 0.1130

10/10 ━━━━━━━━━━━━━━━━━━━━ 45s 4s/step - accuracy: 0.9817 - loss: 0.1101 - val_accuracy: 1.0000 - val_loss: 0.0647 - learning_rate: 0.0010
Epoch 8/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 41s 4s/step - accuracy: 0.9932 - loss: 0.0441 - val_accuracy: 0.9250 - val_loss: 0.1155 - learning_rate: 0.0010
Epoch 9/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9634 - loss: 0.0661
Epoch 9: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
10/10 ━━━━━━━━━━━━━━━━━━━━ 41s 4s/step - accuracy: 0.9656 - loss: 0.0636 - val_accuracy: 0.9750 - val_loss: 0.0767 - learning_rate: 0.0010
Epoch 10/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 39s 4s/step - accuracy: 0.9834 - loss: 0.0455 - val_accuracy: 0.9500 - val_loss: 0.0811 - learning_rate: 2.0000e-04
Epoch 11/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9817 - loss: 0.0496
Epoch 11: ReduceLROnPlateau reducing learning rate to 4.0000001899898055e-05.
10/10 ━━━━━━━━━━━━━━━━━━━━ 39s 4s/step - accuracy: 0.9828 - loss: 0.0481 - val_accuracy: 0.9750

In [16]:
from google.colab import files
uploaded = files.upload()


Saving WhatsApp Image 2025-08-08 at 11.24.03_315daf0d.jpg to WhatsApp Image 2025-08-08 at 11.24.03_315daf0d (1).jpg


In [19]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
from google.colab import files
import os

# Define the image size used for training your model
# IMPORTANT: This must match the size you used in your training code.
IMG_SIZE = (260, 260)

# Step 1: Load your trained model
# The model was saved as "best_model.h5" in the training code.
# The `compile` argument is needed because you're using a custom layer
# that needs to be recompiled when the model is loaded.
try:
    model = load_model('best_model.h5', compile=False)
    print("✅ Model loaded successfully!")
except Exception as e:
    print(f"❌ Error loading model: {e}")
    print("Please make sure you have run the training code and the model file 'best_model.h5' exists.")
    # You might need to change the path if you saved it elsewhere.

# Step 2: Upload the new image for testing
# This will open a file browser dialog for you to select a file from your computer.
# The uploaded image will be placed in the current Colab session's root directory.
print("\nPlease upload the image you want to test.")
uploaded = files.upload()

# Get the name of the uploaded file
file_name = list(uploaded.keys())[0]
image_path = os.path.join('/content', file_name)

# Step 3: Preprocess the new image
def preprocess_image(img_path):
    """
    Loads an image from a path, resizes it, and preprocesses it for the model.
    """
    # Load the image and resize it
    img = image.load_img(img_path, target_size=IMG_SIZE)
    # Convert the image to a numpy array
    img_array = image.img_to_array(img)
    # The model expects a batch of images, so we add a new dimension
    # to convert the shape from (height, width, channels) to (1, height, width, channels)
    img_array = np.expand_dims(img_array, axis=0)
    # Preprocess the image in the same way as the training data
    # We are using EfficientNetV2B2, which has a specific preprocessing function
    return tf.keras.applications.efficientnet_v2.preprocess_input(img_array)

# Preprocess the image
preprocessed_img = preprocess_image(image_path)

# Step 4: Make a prediction
print(f"\nMaking a prediction for '{file_name}'...")
predictions = model.predict(preprocessed_img)

# Step 5: Interpret the prediction
# The model outputs a single value between 0 and 1.
# We need to map this to our class names.
# Based on your data structure, 'COCONUT TREE' is class 0 and 'NOT COCONUT TREE' is class 1.
# You can check this by printing `train_data.class_indices` from your training code.
class_names = ['COCONUT TREE', 'NOT COCONUT TREE']
predicted_class_index = (predictions > 0.5).astype("int32")[0][0]
predicted_class = class_names[predicted_class_index]

print(f"✅ Prediction: {predicted_class}")
print(f"Confidence Score: {predictions[0][0]:.4f}")

# Clean up the uploaded image file
os.remove(image_path)


✅ Model loaded successfully!

Please upload the image you want to test.


Saving WhatsApp Image 2025-08-08 at 11.42.25_a215b24a.jpg to WhatsApp Image 2025-08-08 at 11.42.25_a215b24a.jpg

Making a prediction for 'WhatsApp Image 2025-08-08 at 11.42.25_a215b24a.jpg'...
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step
✅ Prediction: NOT COCONUT TREE
Confidence Score: 0.6341
